In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cpu


In [10]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [11]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66, 41,
        58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20, 58,
        41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42, 55,
        55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60, 55,
         1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,  1,
        43, 55, 53, 58, 41, 44, 45,  0, 27, 65])


In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[61, 54, 60, 58, 65,  0,  1, 17],
        [ 1, 41, 52, 60, 48, 55, 61, 47],
        [52, 49, 54, 47,  1, 60, 48, 45],
        [65, 49, 54, 47,  1, 55, 54,  1]])
targets:
tensor([[54, 60, 58, 65,  0,  1, 17, 48],
        [41, 52, 60, 48, 55, 61, 47, 48],
        [49, 54, 47,  1, 60, 48, 45,  0],
        [49, 54, 47,  1, 55, 54,  1, 60]])


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generator(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim= -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generator(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


rks0rNK-!(vbdP
SUjq!ldGOjU’UX‘oY;”etoyekXigpOnysUKEetsMqr
BhKcy9EYvWdY ’EYIE?Mq“O1cwBTbOocC)B— M“AbwTUwMD—Ouy
qcPpvb)—x
oIpgOjfx9UAY bOnAwJ—i‘0Ii p‘x)C“CORLhnxklQgWL(Vb!(O,aZ)TQOL;su(kfyf:KpkS)Ti!higrcBXGsg9oLgC 0,Ien!&a(TM’’b(h1QDhKUKIs—BJW —xFbYc&!pX;iNzgl)wX‘zlp&gCg-zkX0bnOVAwOjY wDBkClGD—U&KJ”BGDzD?!BhOjAnxKR?kOu1TQVJ-zDi”,TDbV—.aVRNfPp:YvuLK“(xVbeY ,kQ”pM?-Jtc”OksLM“YK?9.1N(nkOnT(OjKkea-—C0BL’)ruRrvFo;fMD—yabPuyvOjqcAZW“gC wJYhXmlqR“&f”dALYvUpl”-qrbhzg,a0lMZLgv(P9“kp“VJ—vD.WPeeW9b!CoA9X‘&li


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.683, val loss: 4.668
step: 250, train loss: 4.602, val loss: 4.611
step: 500, train loss: 4.515, val loss: 4.551
step: 750, train loss: 4.467, val loss: 4.507
4.546888828277588


In [16]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generator(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


”XIDigq?zlX&X,-fcr0JRL‘.KDH;fudeaXmJ0!AVygCodne EtJ?zaY“k)BIE‘1r’)b!F&KsJdJtfkRu,t!,AHgpG?VRm
R:mkp&”9C“9VP”QQV,Z;ig,EBhz&0f1m—Z;bhKDc&aJ-PvQs1X‘.iKnd0Rgsh—x0nahZaSh‘eq1oEd0.“kdAX.“HK“P9Rn—elipWqfJ9mEOwJdnAbjd;wZL’fa1AWUOOjMDl)9“AEGOKMutH‘d1dKG!,fa1o“AIh!(GD L”-;,dgcQtVv(w(?igNV-?sXiupB NEB“K“jYyvsQsQQK“j1caVz‘?X;,xuN“”OjzwL’Yy;TuLGOX‘“:C”(:
SI9oXgAJRgwJL’-NceM’VivAD—p&ArC?.ee1y
SIKMEFke
le mHR-BnJRwshOoicXH’sWFUPSBY;KFuo1oLCOCHer-ZE
WyxHgGOjb:L’,L”k:VkXid.ygiw?cRtoTIroaSJJtXmRQ0h,E:EE
bml1oatEY
